In [3]:
# Importing libraries
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [7]:
# Importing dataset from previous exercise
path=r'C:\Users\aditi\Desktop\Data Analytics_Career Foundry\Data Analytics_Immersion\Project 4\Instacart Basket Analysis'

In [13]:
customers = pd.read_csv(os.path.join(path, 'Data', 'Original Data', 'customers.csv'))

In [15]:
customers.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [18]:
customers.rename(columns = {'First Name' : 'first_name'}, inplace = True)

In [25]:
customers.rename(columns = {'Surnam' : 'last_name'}, inplace = True)
customers.rename(columns = {'STATE' : 'state'}, inplace = True)
customers.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)
customers.rename(columns = {'Gender' : 'gender'}, inplace = True)
customers.rename(columns = {'Age' : 'age'}, inplace = True)

In [27]:
customers.head()

,user_id,first_name,last_name,gender,state,age,date_joined,n_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [32]:
customers=customers.drop(columns=['n_dependants'])

# Data quality and consistency checks

In [39]:
# Checking for missing values
customers.isnull().sum()

user_id               0
first_name        11259
last_name             0
gender                0
state                 0
age                   0
date_joined           0
marital_status        0
income                0
dtype: int64

First name has missing values. Below we can look whether the data can be kept or should be removed.

In [42]:
customers_nan=customers[customers['first_name'].isnull()==True]

In [44]:
customers_nan

,user_id,first_name,last_name,gender,state,age,date_joined,marital_status,income
53,76659,NaN,Gilbert,Male,Colorado,26,1/1/2017,married,41709
73,13738,NaN,Frost,Female,Louisiana,39,1/1/2017,single,82518
82,89996,NaN,Dawson,Female,Oregon,52,1/1/2017,married,117099
99,96166,NaN,Oconnor,Male,Oklahoma,51,1/1/2017,married,155673
105,29778,NaN,Dawson,Female,Utah,63,1/1/2017,married,151819
...,...,...,...,...,...,...,...,...,...
206038,121317,NaN,Melton,Male,Pennsylvania,28,3/31/2020,married,87783
206044,200799,NaN,Copeland,Female,Hawaii,52,4/1/2020,married,108488
206090,167394,NaN,Frost,Female,Hawaii,61,4/1/2020,married,45275
206162,187532,NaN,Floyd,Female,California,39,4/1/2020,single,56325


This data can still be used as 'User ID' is unique. So we need not remove these rows. We can replace NaN by a generic name like "Person".  

In [51]:
customers.fillna({'first_name':'Person'}, inplace=True)

In [53]:
customers.isnull().sum()

user_id           0
first_name        0
last_name         0
gender            0
state             0
age               0
date_joined       0
marital_status    0
income            0
dtype: int64

In [69]:
# Checking for mixed data types
for col in customers.columns.tolist():
  weird = (customers[[col]].map(type) != customers[[col]].iloc[0].apply(type)).any(axis = 1)
  if len (customers[weird]) > 0:
    print (col)

There are no mixed type data in the columns.

In [76]:
# Checking for duplicates
customers_dups = customers[customers.duplicated()]

In [78]:
customers_dups

,user_id,first_name,last_name,gender,state,age,date_joined,marital_status,income


There are no duplicates

In [82]:
# Checking data types of every column
customers.dtypes

user_id            int64
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
marital_status    object
income             int64
dtype: object

In [93]:
# Converting User_id to string
customers['user_id'] = customers['user_id'].astype('str')

In [95]:
customers.describe()

,age,income
count,206209.000000,206209.000000
mean,49.501646,94632.852548
std,18.480962,42473.786988
min,18.000000,25903.000000
25%,33.000000,59874.000000
50%,49.000000,93547.000000
75%,66.000000,124244.000000
max,81.000000,593901.000000


# Combining customers dataset with ords_prods_grouped dataset

In [99]:
ords_prods_grouped=pd.read_pickle(os.path.join(path, 'Data', 'Prepared data', 'ords_prods_grouped.pkl'))

In [104]:
ords_prods_grouped.dtypes

product_id                       int64
product_name                    object
aisle_id                         int64
department_id                    int64
prices                         float64
order_id                         int64
user_id                          int64
order_number                     int64
orders_day_of_week               int64
order_hour_of_day                int64
days_since_prior_order         float64
add_to_cart_order                int64
reordered                        int64
_merge                        category
busiest_days                    object
price_range_loc                 object
busiest_period_of_day           object
max_order                        int64
loyalty_flag                    object
mean_product_price             float64
spending_flag                   object
median_days_between_orders     float64
order_frequency                 object
dtype: object

In [106]:
# Converting User_id which is common in both datasets to string
ords_prods_grouped['user_id'] = ords_prods_grouped['user_id'].astype('str')

In [108]:
ords_prods_grouped.dtypes

product_id                       int64
product_name                    object
aisle_id                         int64
department_id                    int64
prices                         float64
order_id                         int64
user_id                         object
order_number                     int64
orders_day_of_week               int64
order_hour_of_day                int64
days_since_prior_order         float64
add_to_cart_order                int64
reordered                        int64
_merge                        category
busiest_days                    object
price_range_loc                 object
busiest_period_of_day           object
max_order                        int64
loyalty_flag                    object
mean_product_price             float64
spending_flag                   object
median_days_between_orders     float64
order_frequency                 object
dtype: object

In [112]:
ords_prods_grouped.shape

(32404859, 23)

In [114]:
customers.shape

(206209, 9)

In [123]:
customers.dtypes

user_id           object
first_name        object
last_name         object
gender            object
state             object
age                int64
date_joined       object
marital_status    object
income             int64
dtype: object

In [126]:
# combining both datasets
ords_prods_custs = ords_prods_grouped.merge(customers, on = 'user_id', indicator = True)

ValueError: Cannot use name of an existing column for indicator column

In [128]:
#dropping the merge column because it is generating the above error
ords_prods_grouped=ords_prods_grouped.drop(columns=['_merge'])

In [130]:
# combining both datasets
ords_prods_custs = ords_prods_grouped.merge(customers, on = 'user_id', indicator = True)

In [132]:
ords_prods_custs.head()

,product_id,product_name,aisle_id,department_id,prices,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,...,order_frequency,first_name,last_name,gender,state,age,date_joined,marital_status,income,_merge
0,1,Chocolate Sandwich Cookies,61,19,5.8,3139998,138,28,6,11,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,married,49620,both
1,1,Chocolate Sandwich Cookies,61,19,5.8,1977647,138,30,6,17,...,Frequent customer,Charles,Cox,Male,Minnesota,81,8/1/2019,married,49620,both
2,1,Chocolate Sandwich Cookies,61,19,5.8,389851,709,2,0,21,...,Frequent customer,Deborah,Glass,Female,Vermont,66,6/16/2018,married,158302,both
3,1,Chocolate Sandwich Cookies,61,19,5.8,652770,764,1,3,13,...,Frequent customer,Heather,Myers,Female,Wisconsin,40,2/9/2020,married,31308,both
4,1,Chocolate Sandwich Cookies,61,19,5.8,1813452,764,3,4,17,...,Frequent customer,Heather,Myers,Female,Wisconsin,40,2/9/2020,married,31308,both


In [136]:
ords_prods_custs.to_pickle(os.path.join(path, 'Data','Prepared data', 'ords_prods_custs.pkl'))